In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas
import os, shutil, time, pdb, random, pickle
import scipy.stats as stats 

from IPython.display import clear_output

from math import pi
from datetime import datetime
from collections import OrderedDict

import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from elbo import *

# from shapely.geometry import box, mapping
# import descartes

from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

from matplotlib import pyplot as plt
%matplotlib inline
# import seaborn as sns
# sns.set_theme()
pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

import matplotlib
matplotlib.rc_file_defaults()

In [2]:
from plum import dispatch
from varz.torch import Vars
import torch.nn as nn
from stheno.torch import B, GP, EQ, Normal, Measure
from matrix import Diagonal

# Detect device.
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [3]:
from gp_mlp import *

In [4]:
# # Import Beas and Sutlej shapefiles
# beas_shp = '/Users/marron31/Google Drive/PhD/gis/exports/beas_watershed.shp'
# sutlej_shp = '/Users/marron31/Google Drive/PhD/gis/exports/sutlej_watershed.shp'

# beas = geopandas.read_file(beas_shp)
# sutlej = geopandas.read_file(sutlej_shp)

# catchments = beas.append(sutlej)

In [5]:
# Let all of Stheno run on that device.
B.device(device).__enter__()
B.epsilon = 1e-1  # Needs to be relatively high for `float32`s

/data/hpcdata/users/marron31/conda-envs/bcdp/lib/python3.9/site-packages/lab/generic.py:296: DeprecationWarning: The use of `device` to change the active device is deprecated. Please use `on_device` instead.
  warnings.warn(


In [8]:
# Parameters
start="2010-01-01"
end="2020-12-31"

TRAIN_PATH = "../../data/norris/enriched_obs/enriched_langtang_obs_norris_ready.pkl"
# TEST_PATH = "../data/pickle/df_stations_val_all_nonzero_extended.pkl"

In [20]:
st = DataPreprocessing(TRAIN_PATH, start, end)
len(st.st)

/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[series][df[series] < 0] = 0


8678

In [21]:
df = st.st.drop_duplicates()
len(df)

8543

In [22]:
predictant = ['Prec']
predictors = [
              #'Date',
              #'Station',
              #'Prec',
              #'Corrected Station Name', 
              'X', 'Y',
              #'Altitude (m)', 
              'Z', 
              'precip_norris', #'wrf_bc_prcp', 
              #'elev_hr', 
              #'aspect','slope', 
              #'doy', 
              'doy_sin', 'doy_cos', 
              #'wrf_prcp_-1', 'wrf_prcp_-2','wrf_prcp_1', 'wrf_prcp_2', 
              #'Basin', 'lon', 'lat', 
#               'era5_u', 'era5_v',
              #'era5_u_-2', 'era5_u_-1', 'era5_u_1', 'era5_u_2', 
              #'era5_v_-2', 'era5_v_-1', 'era5_v_1', 'era5_v_2'
             ]

In [23]:
# root_folder = '../../data/norris/enriched_obs/enriched_langtang_obs_norris_ready.pkl'

In [28]:
stations_langtang = ['Tipping Bucket Lama Hotel', 
                    'Tipping Bucket Langtang',
                   'Tipping Bucket Kyanjing', 
#                    'Tipping Bucket Numthang old',
#                    'Tipping Bucket Jathang', 
#                     'Pluviometer Yala', 
#                     'AWS Kyangjing',
#                    'AWS Yala BC', 
                    'Tipping Bucket Ganja La 3',
                   'Tipping Bucket Ganja La 2',
                   'Tipping Bucket Langshisha Glacier (next to Pluviometer)',
                   'Tipping Bucket Ganja La 1',
                    'Tipping Bucket Langshisha BC',
                   'Tipping Bucket Shalbachum',
#                    'Pluviometer Langshisha Glacier (off-glacier)',
                   'Pluviometer GanjaLa', 'Pluviometer Morimoto']

exclusion_list = ['Tipping Bucket Numthang old', 'Tipping Bucket Jathang',
                   'Pluviometer Langshisha Glacier (off-glacier)', 'Pluviometer Yala',
                   'AWS Kyangjing', 'AWS Yala BC']

[s not in exclusion_list for s in stations_langtang]

sublist = ['AWS Kyangjing', 'AWS Yala BC', 'Pluviometer Yala']

In [45]:
ds_dataset = UpperIndusDataset(TRAIN_PATH, start, end, predictant, predictors, stations=None)

train_mean = ds_dataset.mean
train_var = ds_dataset.var

dataloader = DataLoader(dataset=ds_dataset, batch_size=16, shuffle=True)

train_dataloader = validation_dataloader = dataloader

/users/marron31/repos/upper-indus-prec-bc/preprocessing_utils.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[series][df[series] < 0] = 0


TypeError: Invalid comparison between dtype=datetime64[ns] and float

In [39]:
model = MLP(in_channels=5, #len(predictors)+1, 
            hidden_channels=[10], 
            likelihood_fn='bgmm', # 'gamma', 'ggmm', bgmm', 'b2gmm', 'b2sgmm'
            dropout_rate=0,
           )

print(f'Number of parameters: {sum(p.numel() for p in model.parameters())}')

Number of parameters: 93


In [40]:
model.to(device)

MLP(
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=False)
  (hidden): ModuleList(
    (0): Linear(in_features=5, out_features=10, bias=True)
  )
  (out): Linear(in_features=10, out_features=3, bias=True)
)

In [41]:
print_progress = True
plot_x_ind = True
validate_flag = False
f_marginal_flag = False
mc_samples = 10

In [42]:
# arr, _ = UpperIndusGridDataset(root_folder, train_mean, train_var).__getitem__(10)

# x_min = arr[0,:].min()
# x_max = arr[0,:].max() 
# y_min = arr[1,:].min()
# y_max = arr[1,:].max()

In [43]:
st = ds_dataset.st
x_ind_stations = st.groupby('Station').mean()[['X','Y']].values
x_ind_stations.shape

(17, 2)

In [44]:
# Number of points for inducing points grid
x_points = 10
y_points = 10

#  Grid of equally spaced points covering the spatial extent for which station data is available
grid = np.meshgrid(np.linspace(st['X'].min(),st['X'].max(),x_points),np.linspace(st['Y'].min(),st['Y'].max(),y_points))
# grid = np.meshgrid(np.linspace(x_min,x_max,x_points),np.linspace(y_min,y_max,y_points))

# Make grid into a tensor
grid = torch.tensor(grid, dtype=torch.float32).flatten(start_dim=1).permute(1,0)

# x_ind = grid.clone().detach().requires_grad_(True)
# num_ind_points = x_points * y_points

# Inducing points are points in the station
x_ind = torch.tensor(x_ind_stations, dtype=torch.float32).detach().requires_grad_(False).to(device)
num_ind_points = len(x_ind)

n = len(st)

train_loss, train_loglik, train_kl = [], [], []
val_loss, val_loglik, val_kl = [], [], []
test_loss, test_loglik, test_kl = [], [], []

train_loss_batch, train_loglik_batch, train_kl_batch = RunningAverage(), RunningAverage(), RunningAverage()
val_loss_batch, val_loglik_batch, val_kl_batch = RunningAverage(), RunningAverage(), RunningAverage()

# model.train()
# prior = Measure()

with Measure() as prior:
    f1 = GP(EQ().stretch(10))
#     f2 = GP(EQ().stretch(2))
#     f3 = GP(EQ().stretch(0.5))
    f = f1

# f = GP(EQ().stretch(0.5)) # f is drawn from a GP
 
q = ApproximatePosterior(num_ind_points) # q is the approximate posterior

# optimizer = torch.optim.Adam(list(model.parameters())+list(q.parameters())+[x_ind], lr=10e-3)
optimizer = torch.optim.Adam(list(model.parameters())+list(q.parameters()), lr=10e-3)

num_epochs = 10

for e in range(num_epochs):
    
    # TRAIN EPOCH
    
    model.train()
        
#     if plot_x_ind and (e % 10 == 0):
#         fig, ax = plt.subplots(figsize=(5,5))
#         ax.scatter(x_ind.detach()[:,0].cpu(),
#                    x_ind.detach()[:,1].cpu()
#                   )
#         plt.show()

    start = time.time()

    train_loss_batch.reset()
    train_loglik_batch.reset()
    train_kl_batch.reset()
    val_loss_batch.reset()
    val_loglik_batch.reset()
    val_kl_batch.reset()

    n = train_dataloader.dataset.n

    for i, (inputs, labels) in enumerate(train_dataloader):

        inputs = inputs.to(device)
        labels = labels.to(device)

        elbo, recon, kl, num_points = forward_backward_pass(inputs, labels, n, model, optimizer, q, f, x_ind, 
                                                            inducing_points=True, backward=True, f_marginal=f_marginal_flag, n_samples=mc_samples)

        # Keep track of loss terms
        train_loss_batch.update(-elbo.item())
        train_loglik_batch.update(-recon.item()/num_points)
        train_kl_batch.update(kl.item())

    elapsed = time.time() - start

    if print_progress:
        print(f'Average values for training epoch {e}: -elbo: {train_loss_batch.avg:.4f} | kl: {train_kl_batch.avg:.4f} | -log-lik: {train_loglik_batch.avg:.4f} -- time elapsed: {elapsed:.2f}')
    
    # Add average batch loss terms to lists
    train_loss.append(train_loss_batch.avg)
    train_loglik.append(float(train_loglik_batch.avg.cpu()))
    train_kl.append(train_kl_batch.avg)
    
    if validate_flag:

        # VALIDATION EPOCH

        model.eval()

        n = validation_dataloader.dataset.n
        
        start = time.time()

        with torch.no_grad():

            for i, (inputs, labels) in enumerate(validation_dataloader):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                elbo, recon, kl, num_points = forward_backward_pass(inputs, labels, n, model, optimizer, q, f, x_ind, 
                                                                    inducing_points=True, backward=False, f_marginal=f_marginal_flag, n_samples=mc_samples)

                # Keep track of loss terms
                val_loss_batch.update(-elbo.item())
                val_loglik_batch.update(-recon.item()/num_points)
                val_kl_batch.update(kl.item())

        elapsed_val = time.time() - start

        if print_progress:
            print(f'Average values for validation epoch {e}: -elbo: {np.mean(val_loss_batch):.4f} | kl: {np.mean(val_kl_batch):.4f} | -log-lik: {np.mean(val_loglik_batch):.4f} -- time elapsed: {elapsed_val:.2f}')

        # Add average batch loss terms to lists
        val_loss.append(val_loss_batch.avg)
        val_loglik.append(val_loglik_batch.avg)
        val_kl.append(val_kl_batch.avg)

/tmp/ipykernel_12065/3299178086.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1656352660876/work/torch/csrc/utils/tensor_new.cpp:201.)
  grid = torch.tensor(grid, dtype=torch.float32).flatten(start_dim=1).permute(1,0)


ValueError: Dimensions {'Date'} do not exist. Expected one or more of Frozen({'index': 8543})

In [ ]:
train_vars = [train_loss, train_loglik, train_kl] 
validation_vars = [val_loss, val_loglik, val_kl]

labels = ['-elbo','-loglik','kldiv']

fig, axes = plt.subplots(1,3, figsize=(20,5))
for i,ax in enumerate(axes.flatten()):
    ax.plot(train_vars[i], label='train')
    ax.plot(validation_vars[i], label='val')
    ax.legend()
    ax.set_title(f'{labels[i]} | best: {np.min(train_vars[i]):.2f}')
plt.tight_layout()

In [ ]:
test_dataloader = DataLoader(dataset=UpperIndusGridDataset(root_folder, train_mean, train_var), batch_size=1, shuffle=True)

inducing_points = True

model.eval()

with torch.no_grad():
    
    for index, (inputs, inputs_plot) in enumerate(test_dataloader):
        
        b = inputs.shape[0]
        
        if index == 0:
        
            q.build_normal()

            x = inputs[0,:2,:].permute(1,0).float()
            
            inputs = inputs[:,2:,:]

            # Sample z and concatenate to inputs
            if inducing_points:
                
                #f_sample = f_post(x).sample(b).permute(1,0).unsqueeze(1)
                
                f_post = f | (f(x_ind), q.sample())
                
                if f_marginal_flag:
                    f_sample = Normal(f_post.mean(x),
                                      Diagonal(f_post.kernel.elwise(x)[:, 0])
                                      ).sample().permute(1,0).unsqueeze(1)
                else:
                    f_sample = f_post(x).sample().permute(1,0).unsqueeze(1)
                
                inputs = torch.cat([f_sample, inputs], dim=1)
                 
            else:
                q_sample = q.sample(b).permute(1,0).unsqueeze(1)
                
                inputs = torch.cat([q_sample, inputs], dim=1)

            # Masking for missing data
            inputs = inputs.permute(0,2,1)
 
            mask = ~torch.any(inputs.isnan(),dim=2)
            k = mask.sum()

            # Forward pass
            outputs = model(inputs[mask].float())

In [ ]:
col_names = ['X','Y','Z','wrf_prcp','doy_sin','doy_cos','GPsample','pi','alpha','beta']
#col_names = ['X','Y','Z','wrf_prcp','doy_sin','doy_cos','GPsample','alpha','beta']

data = np.concatenate([inputs_plot.squeeze().permute(1,0).numpy(),
                       inputs[:,:,0].permute(1,0),
                       outputs.numpy()], axis=1)

df = pd.DataFrame(data, columns=col_names)

gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.X, df.Y))

gdf['uniform'] = gdf.apply(lambda x: np.random.uniform(0,1),axis=1)
#gdf['uniform'] = np.random.uniform(0,1)

gdf['sample'] = gdf.apply(sample, axis=1)

gdf['uniform'] = gdf.apply(lambda x: 0.5, axis=1)
gdf['bg_median'] = gdf.apply(sample, axis=1)

gdf['g_mean'] = gdf['alpha']/gdf['beta']

gdf['p'] = 1 - gdf['pi']

x_ind_plot = x_ind.detach().numpy() * np.sqrt(train_var[:2]) + train_mean[:2] 

In [ ]:
st_stations = st.groupby('Station').mean()[['X','Y']] * np.sqrt(train_var[:2]) + train_mean[:2]

In [ ]:
gdf_stations = geopandas.GeoDataFrame(geometry=geopandas.points_from_xy(x_ind_plot[:,0], x_ind_plot[:,1]))

gdf_stations['q_sample'] = q.sample().detach().numpy()
gdf_stations.plot('q_sample', cmap='viridis')

# plt.plot(gdf_stations['geometry'].x, gdf_stations['q_sample'], 'o')

In [ ]:
variables = ['wrf_prcp',
             'sample',
             'p',
             'g_mean',
             #'bg_median',
             'Z',
             'GPsample']

labels = ['RCM output (raw)', 
          'BGMM Sample (mm/day)', 
          'Probability of rain',
          'BGMM Mean (slab component) (mm/day)',
          #'BGMM Median (mm/day)',
          'Elevation (m.a.s.l.)',
          'GP Sample']

cmaps = ['terrain',
         'terrain',
         'terrain',
         'terrain',
         #'Blues',
         'terrain',
         'terrain']

fig, axes = plt.subplots(3,2,figsize=(15,15))

for idx, ax in enumerate(axes.flatten()):
    
    if idx < len(variables):
    
    #     gdf.set_crs(catchments.crs)
    #     gdf_clip = geopandas.clip(gdf, catchments)
    
        gdf.plot(variables[idx], ax=ax, legend=True, cmap=cmaps[idx], legend_kwds={'shrink': 0.5})
        beas.plot(ax=ax, facecolor='None', edgecolor='black')
        #gdf_stations.plot(ax=ax, color='red')
        #sutlej.plot(ax=ax, facecolor='None', edgecolor='black')

        ax.set_title(labels[idx])
        
        #if variables[idx]=='GPsample':
            #ax.scatter(x_ind_plot[:,0],x_ind_plot[:,1], s=4, c='r')
            #ax.scatter(st_stations['X'],st_stations['Y'], s=3, c='w')

    ax.set_frame_on(False)
    ax.set_yticks([])
    ax.set_xticks([])     

plt.tight_layout()